### Load Packages

In [1]:
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf
import cv2

## pre-precessing

In [20]:
def iris_mask(img):
    img_org_gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    img_histEqualization= cv2.equalizeHist(img_org_gray)
    _,img_binary= cv2.threshold(img_org_gray,70,255,cv2.THRESH_BINARY)
    center_pupil = cv2.HoughCircles(img_binary, cv2.HOUGH_GRADIENT,1,250,
                           param1=80,param2=10,minRadius=30,
                           maxRadius=70)
    center_pupil = np.uint16(np.around(center_pupil))
    draw = img.copy()
#     for i in center_pupil[0,:]:
#         print(i,i[0],i[1],i[2])
#         draw = cv2.circle(img,(i[0],i[1]),i[2],(255,0,0),2)
#         draw = cv2.circle(draw,(i[0],i[1]),2,(255,0,0),3)
    (x1, y1) = center_pupil[0][0][0], center_pupil[0][0][1]
    circles = cv2.HoughCircles(img_histEqualization, cv2.HOUGH_GRADIENT,1,250,
                           param1=80,param2=30,minRadius=50,
                           maxRadius=100)
    circles = np.uint16(np.around(circles))
    draw_shift = draw.copy()     
    for i in circles[0,:]:       
        print(i,i[0],i[1],i[2])
        mask = np.zeros_like(img)
        mask = cv2.circle(mask, (x1,y1), i[2], (255,255,255), -1)
        draw_shift = cv2.circle(draw_shift,(x1,y1),i[2],(255,0,0),2)
        #draw_shift = cv2.circle(draw_shift,(x1,y1),2,(255,0,0),3)  
        draw_mask = cv2.bitwise_and(draw_shift, mask)
    return draw_mask

### Prepare Data for Training and Testing

In [ ]:
x_train, y_train = np.zeros([2688, 224, 224,3]), np.zeros([2688])
x_test, y_test = np.zeros([896, 224, 224,3]), np.zeros([896])

folder = "IITD_database/people"
i = 0
for people in range(1,225):
    person_folder = folder + "/" + "0"*(3-len(str(people))) + str(people)
    file_names = os.listdir(person_folder)
    for num_image in range(1,7):
        path = person_folder + "/" + file_names[num_image-1]
        image = Image.open(path)
        image = image.resize((224,224))
        image = np.asarray(image)
        image = iris_mask(image)
        #x = image.reshape((1,224,224,3))
        row_num = i*6 + num_image -1
        x_train[row_num,:,:,:] = image
        y_train[row_num] = people
        
        flip = np.flip(x_train[i],1)
        flip = np.resize(flip, (1,224,224,3))
        x_train[row_num+1344,:,:,:] = flip
        y_train[row_num+1344] = people
    i += 1
    
i = 0
for people in range(1,225):
    person_folder = folder + "/" + "0"*(3-len(str(people))) + str(people)
    file_names = os.listdir(person_folder)
    for num_image in range(7,11):
        path = person_folder + "/" + file_names[num_image-1]
        image = Image.open(path)
        image = image.resize((224,224))
        image = np.asarray(image)
        image = iris_mask(image)
        #x = image.reshape((1,224,224,3))
        row_num = i*4 + num_image -7
        x_test[row_num,:,:,:] = image
        y_test[row_num] = people
    i += 1
    
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [23]:
index_train = np.random.permutation(2688)
index_test = np.random.permutation(896)
x_train, y_train = x_train[index_train], y_train[index_train]
x_test, y_test = x_test[index_test], y_test[index_test]

In [24]:
x_train, x_test = tf.cast(x_train, tf.float32)/255.0, tf.cast(x_test, tf.float32)/255.0
y_train, y_test = tf.one_hot(y_train-1, depth=224), tf.one_hot(y_test-1, depth=224)
#y_train, y_test = tf.cast(y_train, tf.float32), tf.cast(y_test, tf.float32)

### Model Definition

In [25]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Model

In [26]:
from tensorflow.keras import Model
pre_trained_model = ResNet50(input_shape=(224,224,3), weights='imagenet')
ResNet = Model(inputs=pre_trained_model.input,outputs=pre_trained_model.layers[-2].output)
#ResNet.summary()

# First time, train without adjusting the ResNet weights to train the final dense layer first
ResNet.trainable =  False
model = tf.keras.Sequential([
                             ResNet,
                             tf.keras.layers.Dense(224, activation='softmax')
                                      ])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 224)               458976    
Total params: 24,046,688
Trainable params: 458,976
Non-trainable params: 23,587,712
_________________________________________________________________


In [27]:
model.add_loss(lambda: 0.01*tf.reduce_sum(tf.square(model.layers[-1].kernel)))

model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=tf.keras.metrics.CategoricalAccuracy())

In [28]:
model.fit(x_train, y_train, batch_size=24, epochs=1, validation_split=0.2)

90/90 [==============================] - 35s 243ms/step - loss: 9.0159 - categorical_accuracy: 0.0051 - val_loss: 7.1276 - val_categorical_accuracy: 0.0019


In [29]:
ResNet.trainable =  True
model.add_loss(lambda: 0.01*tf.reduce_sum(tf.square(model.layers[-1].kernel)))

model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=tf.keras.metrics.CategoricalAccuracy())

model.fit(x_train, y_train, batch_size=24, epochs=15, validation_split=0.2)

Epoch 1/15
90/90 [==============================] - 61s 600ms/step - loss: 7.9444 - categorical_accuracy: 0.1652 - val_loss: 7.9459 - val_categorical_accuracy: 0.0019
Epoch 2/15
90/90 [==============================] - 52s 578ms/step - loss: 4.2000 - categorical_accuracy: 0.7466 - val_loss: 7.5499 - val_categorical_accuracy: 0.0093
Epoch 3/15
90/90 [==============================] - 52s 576ms/step - loss: 2.4172 - categorical_accuracy: 0.9733 - val_loss: 7.1274 - val_categorical_accuracy: 0.0056
Epoch 4/15
90/90 [==============================] - 52s 576ms/step - loss: 1.7100 - categorical_accuracy: 0.9975 - val_loss: 6.7218 - val_categorical_accuracy: 0.0037
Epoch 5/15
90/90 [==============================] - 52s 577ms/step - loss: 1.2722 - categorical_accuracy: 0.9997 - val_loss: 6.3804 - val_categorical_accuracy: 0.0074
Epoch 6/15
90/90 [==============================] - 52s 577ms/step - loss: 0.9752 - categorical_accuracy: 1.0000 - val_loss: 6.0908 - val_categorical_accuracy: 0.018

In [30]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(x_train, y_train, batch_size=24, epochs=20, validation_split=0.2, callbacks=[early_stop])

Epoch 1/20
90/90 [==============================] - 52s 578ms/step - loss: 0.6264 - categorical_accuracy: 0.9572 - val_loss: 1.3952 - val_categorical_accuracy: 0.8030
Epoch 2/20
90/90 [==============================] - 52s 576ms/step - loss: 0.3605 - categorical_accuracy: 0.9986 - val_loss: 0.6803 - val_categorical_accuracy: 0.9349
Epoch 3/20
90/90 [==============================] - 52s 576ms/step - loss: 0.2684 - categorical_accuracy: 1.0000 - val_loss: 0.5237 - val_categorical_accuracy: 0.9572
Epoch 4/20
90/90 [==============================] - 52s 577ms/step - loss: 0.2325 - categorical_accuracy: 0.9991 - val_loss: 0.5121 - val_categorical_accuracy: 0.9480
Epoch 5/20
90/90 [==============================] - 52s 576ms/step - loss: 0.1982 - categorical_accuracy: 1.0000 - val_loss: 0.4729 - val_categorical_accuracy: 0.9461
Epoch 6/20
90/90 [==============================] - 52s 576ms/step - loss: 0.1955 - categorical_accuracy: 0.9977 - val_loss: 0.5134 - val_categorical_accuracy: 0.940

In [31]:
model.evaluate(x_test, y_test)

28/28 [==============================] - 8s 207ms/step - loss: 0.4521 - categorical_accuracy: 0.9420


[0.45205608010292053, 0.9419642686843872]